# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298550623859                   -0.85    5.0
  2   -8.300209526834       -2.78       -1.26    1.8
  3   -8.300447338513       -3.62       -1.87    2.8
  4   -8.300461351068       -4.85       -2.72    1.1
  5   -8.300464070983       -5.57       -3.08    6.8
  6   -8.300464350547       -6.55       -3.22    7.6
  7   -8.300464495001       -6.84       -3.37    1.0
  8   -8.300464567225       -7.14       -3.49    1.1
  9   -8.300464629827       -7.20       -3.73    1.0
 10   -8.300464637371       -8.12       -3.86    1.0
 11   -8.300464643091       -8.24       -4.23    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.63717260814                   -0.71    6.1
  2   -16.64122596221       -2.39       -1.15    1.0
  3   -16.67846587523       -1.43       -1.76    3.0
  4   -16.67924843328       -3.11       -2.25    3.8
  5   -16.67928148086       -4.48       -2.64    6.4
  6   -16.67928617598       -5.33       -3.19    1.1
  7   -16.67928620972       -7.47       -3.40    3.6
  8   -16.67928621335       -8.44       -4.08    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32742675476                   -0.56    6.6
  2   -33.33445146411       -2.15       -1.00    1.0
  3   -33.33591824682       -2.83       -1.73    7.5
  4   -33.33688895954       -3.01       -2.43    5.0
  5   -33.33694168414       -4.28       -3.06    7.5
  6   -33.33694387885       -5.66       -3.81    4.4
  7   -33.33694392482       -7.34       -4.32    6.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298449881500                   -0.85    5.1
  2   -8.300246589016       -2.75       -1.58    1.0
  3   -8.300377273097       -3.88       -2.25    4.5
  4   -8.300309081455   +   -4.17       -2.16   12.4
  5   -8.300464271778       -3.81       -3.61    1.6
  6   -8.300464590947       -6.50       -3.89    4.0
  7   -8.300464631615       -7.39       -4.22    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32571846879                   -0.56    7.4
  2   -33.32202567510   +   -2.43       -1.27    1.1
  3   -19.86661434309   +    1.13       -0.50    9.5
  4   -33.30078453408        1.13       -1.71    7.5
  5   -33.27485814953   +   -1.59       -1.35    3.4
  6   -33.25634586184   +   -1.73       -1.61    3.5
  7   -33.18685288718   +   -1.16       -1.47    4.2
  8   -33.33595909667       -0.83       -2.23    3.8
  9   -33.33459413473   +   -2.86       -2.27    3.0
 10   -33.33681933045       -2.65       -2.78    2.2
 11   -33.33691221566       -4.03       -3.18    3.5
 12   -33.33694192555       -4.53       -3.57    3.1
 13   -33.33694336389       -5.84       -3.83    3.1
 14   -33.33694388166       -6.29       -4.15    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.